## Start 

In [84]:
from pyspark.sql import SparkSession

#Initialize the session 
spark=SparkSession.builder.appName("Preparation").getOrCreate()

#Read the data
data="./data/Data.csv" #Chemin vers le fichier
df=spark.read.csv(data,header=True,inferSchema=True)


In [85]:
#Show the data
df.show()
#Number of rows
df.count()

+-----------------+---------------+--------+-----------+--------------------+--------+------------+--------+-----------+--------------------+-----+
|          Airline|Date_of_Journey|  Source|Destination|               Route|Dep_Time|Arrival_Time|Duration|Total_Stops|     Additional_Info|Price|
+-----------------+---------------+--------+-----------+--------------------+--------+------------+--------+-----------+--------------------+-----+
|           IndiGo|     24/03/2019|Banglore|  New Delhi|           BLR → DEL|   22:20|01:10 22 Mar|  2h 50m|   non-stop|             No info| 3897|
|        Air India|      1/05/2019| Kolkata|   Banglore|CCU → IXR → BBI →...|   05:50|       13:15|  7h 25m|    2 stops|             No info| 7662|
|      Jet Airways|      9/06/2019|   Delhi|     Cochin|DEL → LKO → BOM →...|   09:25|04:25 10 Jun|     19h|    2 stops|             No info|13882|
|           IndiGo|     12/05/2019| Kolkata|   Banglore|     CCU → NAG → BLR|   18:05|       23:30|  5h 25m|    

10683

In [86]:
df.select("Total_Stops").distinct().show()

+-----------+
|Total_Stops|
+-----------+
|    4 stops|
|   non-stop|
|    2 stops|
|     1 stop|
|    3 stops|
|       NULL|
+-----------+



## Clean the data 

In [87]:
df.printSchema()
# #Data format 
#  |-- Airline: string (nullable = true)
#  |-- Date_of_Journey: string (nullable = true)
#  |-- Source: string (nullable = true)
#  |-- Destination: string (nullable = true)
#  |-- Route: string (nullable = true)
#  |-- Dep_Time: timestamp (nullable = true)
#  |-- Arrival_Time: string (nullable = true)
#  |-- Duration: string (nullable = true)
#  |-- Total_Stops: string (nullable = true)
#  |-- Additional_Info: string (nullable = true)
#  |-- Price: integer (nullable = true)
#It is neccesary to change few format

root
 |-- Airline: string (nullable = true)
 |-- Date_of_Journey: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- Destination: string (nullable = true)
 |-- Route: string (nullable = true)
 |-- Dep_Time: string (nullable = true)
 |-- Arrival_Time: string (nullable = true)
 |-- Duration: string (nullable = true)
 |-- Total_Stops: string (nullable = true)
 |-- Additional_Info: string (nullable = true)
 |-- Price: integer (nullable = true)



In [88]:
from pyspark.sql.functions import (
    col,  
    to_timestamp, 
    when, 
    date_format
)

# Configurer Spark pour utiliser le mode 'LEGACY' qui accepte les formats de date anciens
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")


# 1. Convert Date_of_Journey to timestamp
# Format: dd/MM/yyyy to timestamp
df = df.withColumn("Date_of_Journey", date_format(to_timestamp(col("Date_of_Journey"), "dd/MM/yyyy"), "yyyy-MM-dd"))
df=df.withColumn("Day", date_format(to_timestamp(col("Date_of_Journey"), "yyyy-MM-dd"), "dd"))
df=df.withColumn("Month", date_format(to_timestamp(col("Date_of_Journey"), "yyyy-MM-dd"), "MM"))
df=df.withColumn("Year", date_format(to_timestamp(col("Date_of_Journey"), "yyyy-MM-dd"), "yyyy"))

# 2. Process Dep_Time

df = df.withColumn('Dep_Time', date_format(col('Dep_Time'), 'HH:mm'))

df.printSchema()
df.show()

root
 |-- Airline: string (nullable = true)
 |-- Date_of_Journey: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- Destination: string (nullable = true)
 |-- Route: string (nullable = true)
 |-- Dep_Time: string (nullable = true)
 |-- Arrival_Time: string (nullable = true)
 |-- Duration: string (nullable = true)
 |-- Total_Stops: string (nullable = true)
 |-- Additional_Info: string (nullable = true)
 |-- Price: integer (nullable = true)
 |-- Day: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- Year: string (nullable = true)

+-----------------+---------------+--------+-----------+--------------------+--------+------------+--------+-----------+--------------------+-----+---+-----+----+
|          Airline|Date_of_Journey|  Source|Destination|               Route|Dep_Time|Arrival_Time|Duration|Total_Stops|     Additional_Info|Price|Day|Month|Year|
+-----------------+---------------+--------+-----------+--------------------+--------+------------+-----

In [89]:
from pyspark.sql.functions import udf, col
from pyspark.sql.types import StringType

# Function to clean and extract only the hour
def clean_arrival_time(arrival_time):
    """
    Extract the hour part (HH:mm) from the Arrival_Time.
    Args:
        arrival_time (str): Raw arrival time.
    Returns:
        str: Cleaned time in HH:mm format.
    """
    if not arrival_time:
        return None

    # If the time contains both hour and date (like '01:10 22 Mar|'), split and get the hour part
    if " " in arrival_time:
        return arrival_time.split(" ")[0]  # Get only the time part before the space (HH:mm)
    else:
        return arrival_time  # Already in HH:mm format

# Create a UDF to apply the function
clean_arrival_time_udf = udf(clean_arrival_time, StringType())

# Apply the transformation on the DataFrame to extract the hour part
df = df.withColumn(
    "Arrival_Time", 
    clean_arrival_time_udf(col("Arrival_Time"))  # Apply UDF to extract the time
)

# Show the schema and the DataFrame data
df.printSchema()
df.show(truncate=False)


root
 |-- Airline: string (nullable = true)
 |-- Date_of_Journey: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- Destination: string (nullable = true)
 |-- Route: string (nullable = true)
 |-- Dep_Time: string (nullable = true)
 |-- Arrival_Time: string (nullable = true)
 |-- Duration: string (nullable = true)
 |-- Total_Stops: string (nullable = true)
 |-- Additional_Info: string (nullable = true)
 |-- Price: integer (nullable = true)
 |-- Day: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- Year: string (nullable = true)

+-----------------+---------------+--------+-----------+---------------------+--------+------------+--------+-----------+---------------------------+-----+---+-----+----+
|Airline          |Date_of_Journey|Source  |Destination|Route                |Dep_Time|Arrival_Time|Duration|Total_Stops|Additional_Info            |Price|Day|Month|Year|
+-----------------+---------------+--------+-----------+---------------------+--------+-

In [90]:
from pyspark.sql.functions import udf, col
from pyspark.sql.types import FloatType

# 4. Process Duration
# Convert duration from 'Xh Ym' format to hours
def duration_to_hours(duration):
    """
    Convert flight duration to hours.
    Args:
        duration (str): Duration in format 'Xh Ym' or 'Xh' or 'Ym'.
    Returns:
        float: Total duration in hours.
    """
    if not duration:
        return None
    
    total_hours = 0.0
    
    # Process hours and minutes
    if "h" in duration:
        hours = float(duration.split("h")[0].strip())
        total_hours += hours  # Add hours directly
        if "m" in duration:
            minutes = int(duration.split("h")[1].split("m")[0].strip())
            total_hours += minutes / 60.0  # Convert minutes to hours and add
    elif "m" in duration:
        minutes = int(duration.replace("m", "").strip())
        total_hours += minutes / 60.0  # Convert minutes to hours
    
    return total_hours

# Create a UDF to apply the function
duration_to_hours_udf = udf(duration_to_hours, FloatType())

# Apply the transformation on the DataFrame to convert duration to hours
df = df.withColumn("Duration", duration_to_hours_udf(col("Duration")))

# Show the schema and the DataFrame data
df.printSchema()
df.show(truncate=False)


root
 |-- Airline: string (nullable = true)
 |-- Date_of_Journey: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- Destination: string (nullable = true)
 |-- Route: string (nullable = true)
 |-- Dep_Time: string (nullable = true)
 |-- Arrival_Time: string (nullable = true)
 |-- Duration: float (nullable = true)
 |-- Total_Stops: string (nullable = true)
 |-- Additional_Info: string (nullable = true)
 |-- Price: integer (nullable = true)
 |-- Day: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- Year: string (nullable = true)

+-----------------+---------------+--------+-----------+---------------------+--------+------------+---------+-----------+---------------------------+-----+---+-----+----+
|Airline          |Date_of_Journey|Source  |Destination|Route                |Dep_Time|Arrival_Time|Duration |Total_Stops|Additional_Info            |Price|Day|Month|Year|
+-----------------+---------------+--------+-----------+---------------------+--------+

In [91]:
# 5. Process Total_Stops
# Convert text descriptions to numeric values
df = df.withColumn(
    "Total_Stops", 
    when(col("Total_Stops").contains("non-stop"), 0)
    .when(col("Total_Stops").contains("1 stop"), 1)
    .when(col("Total_Stops").contains("2 stops"), 2)
    .when(col("Total_Stops").contains("3 stops"), 3)
    .when(col("Total_Stops").contains("4 stops"), 4)
    .otherwise(None)  # Use None instead of 0 for unknown cases
)

# 6. Add data validations
# Remove rows with null values in critical columns
df = df.filter(
    (col("Duration").isNotNull()) &
    (col("Dep_Time").isNotNull()) &
    (col("Arrival_Time").isNotNull())
)


#Drop the route 
df=df.drop("Route")
# Display schema and sample data
df.printSchema()


root
 |-- Airline: string (nullable = true)
 |-- Date_of_Journey: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- Destination: string (nullable = true)
 |-- Dep_Time: string (nullable = true)
 |-- Arrival_Time: string (nullable = true)
 |-- Duration: float (nullable = true)
 |-- Total_Stops: integer (nullable = true)
 |-- Additional_Info: string (nullable = true)
 |-- Price: integer (nullable = true)
 |-- Day: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- Year: string (nullable = true)



In [92]:
df.show()

+-----------------+---------------+--------+-----------+--------+------------+---------+-----------+--------------------+-----+---+-----+----+
|          Airline|Date_of_Journey|  Source|Destination|Dep_Time|Arrival_Time| Duration|Total_Stops|     Additional_Info|Price|Day|Month|Year|
+-----------------+---------------+--------+-----------+--------+------------+---------+-----------+--------------------+-----+---+-----+----+
|           IndiGo|     2019-03-24|Banglore|  New Delhi|   22:20|       01:10|2.8333333|          0|             No info| 3897| 24|   03|2019|
|        Air India|     2019-05-01| Kolkata|   Banglore|   05:50|       13:15|7.4166665|          2|             No info| 7662| 01|   05|2019|
|      Jet Airways|     2019-06-09|   Delhi|     Cochin|   09:25|       04:25|     19.0|          2|             No info|13882| 09|   06|2019|
|           IndiGo|     2019-05-12| Kolkata|   Banglore|   18:05|       23:30|5.4166665|          1|             No info| 6218| 12|   05|2019|

In [93]:
#Delete duplicates
df = df.distinct()
df.count()

10461

## Pretreatment of data